In [9]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 204
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [10]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_ConvStudy2_25082023'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy2_25082023'.


{ Session Count = 8; Grid Count = 8; Path = \\dc1\userspace\sebastian\cluster\XESF_BowShock_ConvStudy2_25082023 }

In [11]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2023-08-25 10:44:42Z

In [12]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:24 AM	041bb15f...
#1: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux1-FphiType0-NewAgglo	8/25/2023 10:45:43 AM	06307530...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux1-FphiType0-NewAgglo	8/25/2023 10:45:28 AM	07cf37ee...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...
#4: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:43:47 AM	ff833be6...
#5: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux1-FphiType0-NewAgglo	8/25/2023 10:45:02 AM	ebca118d...
#6: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux1-FphiType0-NewAgglo*	8/25/2023 10:45:51 AM	9466aced...
#7: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	8/25/2023 10:44:42 AM	1ae3733e...


In [13]:
var si=sessions.Pick(1);

In [14]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

041bb15f-4d2b-4c7f-9548-3bf015e2e408

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [15]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [16]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [35]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
sessions=sessions.Where(s => (s.Name.Contains("iProb0")  &&s.Name.Contains("10x32")&&s.Name.Contains("iFlux0"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...


# A Helper Function to obtain different plot formats

In [36]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [38]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M477.9,57.1 L531.3,57.1 M45.6,146.1 L51.1,130.9 L56.6,134.9 L62.2,135.3 L67.7,135.7 L73.2,136.1
 L78.7,141.7 L84.3,143.9 L89.8,141.8 L95.3,142.9 L100.8,143.9 L106.3,146.1 L111.9,146.3 L117.4,146.4
 L122.9,146.4 L128.4,146.4 L134.0,146.3 L139.5,146.3 L145.0,146.3 L150.5,146.3 L156.1,146.3 L161.6,146.4
 L167.1,146.4 L172.6,146.5 L178.1,146.4 L183.7,146.3 L189.2,146.4 L194.7,146.3 L200.2,146.3 L205.8,146.3
 L211.3,146.3 L216.8,146.3 L222.3,146.3 L227.8,146.3 L233.4,146.3 L238.9,155.1 L244.4,159.1 L249.9,175.3
 L255.5,154.0 L261.0,167.5 L266.5,158.9 L272.0,207.4 L277.5,279.9 L283.1,312.5 L288.6,312.1 L294.1,312.9
 L299.6,312.8 L305.2,313.5 L310.7,313.0 L316.2,313.6 L321.7,312.6 L327.2,313.5 L332.8,312.7 L338.3,313.2
 L343.8,313.3 L349.3,313.3 L354.9,313.1 L360.4,313.2 L365.9,313.2 L371.4,313.2 L377.0,313.2 L382.5,313.0
 L388.0,313.1 L393.5,313.1 L399.0,313.2 L404.6,313.1 L410.1,313.1 L415.6,313.1 L421.1,313.1 L426.7,313.0
 L432.2,313.1 L437.7,313.1 L443.2,313.1 L448.7,313.0 L454.3,313.0 L459.8,313.1 L465.3,313.1 L470.8,313.0
 L476.4,313.0 L481.9,313.0 L487.4,312.9 L492.9,313.0 L498.4,313.0 L504.0,313.0 L509.5,313.0 L515.0,313.0
 L520.5,313.0 L526.1,312.9 L531.6,313.0 L537.1,313.0 L542.6,313.0 L548.1,313.0 L553.7,313.0 L559.2,313.0
 L564.7,313.0 L570.2,313.0 L575.8,313.0 L581.3,313.0 L586.8,313.0 L592.3,313.0 L597.9,313.0 L603.4,313.0
 L608.9,312.9 L614.4,313.0 L619.9,313.0 L625.5,311.4 L631.0,313.0 L636.5,313.0 L642.0,313.0 L647.6,313.0
 L653.1,313.0 L658.6,391.7 L664.1,393.0 L669.6,402.7 L675.2,402.7 L680.7,400.3 L686.2,400.4 L691.7,401.9
 L697.3,403.0 L702.8,402.9 L708.3,402.5 L713.8,403.3 L719.3,403.3 L724.9,402.9 L730.4,402.7 L735.9,402.7
 L741.4,402.6 L747.0,402.5 L752.5,402.9 L758.0,402.9 L763.5,403.0 L769.0,403.0 L774.6,403.0 L780.1,403.1
 L785.6,403.1 L791.1,403.1 L796.7,403.1 L802.2,403.1 L807.7,403.1 L813.2,403.1 L818.8,403.1 L824.3,403.1
 L829.8,403.1 L835.3,472.2 L840.8,474.3 L846.4,474.3 L851.9,473.8 L857.4,474.1 L862.9,474.1 L868.5,473.9
 L874.0,473.9 L879.5,473.9 L885.0,473.8 L890.5,473.8 L896.1,473.8 L901.6,473.9 L907.1,473.9 L912.6,473.8
 L918.2,473.9 L923.7,473.9 L929.2,473.8 L934.7,473.8 L940.2,473.8 L945.8,473.8 L951.3,473.8 L956.8,473.8
 L962.3,473.8 L967.9,473.8 L973.4,473.8 L978.9,473.8 L984.4,473.8 L989.9,473.8 L995.5,473.8 L1001.0,473.8
 L1006.5,473.8 L1012.0,473.8 L1017.6,473.8 L1023.1,473.8 L1028.6,473.8 L1034.1,473.8 L1039.7,473.8 L1045.2,473.8
 L1050.7,473.8 L1056.2,473.8 L1061.7,473.8 L1067.3,473.8 L1072.8,473.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M477.9,81.1 L531.3,81.1 M45.6,146.1 L51.1,130.9 L56.6,134.9 L62.2,135.3 L67.7,135.7 L73.2,136.1
 L78.7,141.

## Residual History

In [61]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions.Where(si => si.Name.Contains("iFlux0"))){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 ||r|| - XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M0.0,57.1 L42.5,57.1 M45.6,94.1 L49.1,104.5 L52.6,109.5 L56.2,109.7 L59.7,110.2 L63.2,109.7
 L66.7,121.7 L70.3,134.0 L73.8,140.2 L77.3,147.6 L80.8,148.3 L84.3,159.5 L87.9,186.8 L91.4,196.1
 L94.9,205.2 L98.4,213.4 L102.0,203.6 L105.5,216.9 L109.0,216.1 L112.5,233.3 L116.1,202.5 L119.6,235.7
 L123.1,205.2 L126.6,216.9 L130.1,230.8 L133.7,240.9 L137.2,208.8 L140.7,245.5 L144.2,209.7 L147.8,218.5
 L151.3,228.1 L154.8,241.5 L158.3,214.5 L161.8,221.9 L165.4,213.7 L168.9,110.6 L172.4,114.8 L175.9,104.3
 L179.5,128.3 L183.0,146.7 L186.5,141.2 L190.0,201.5 L193.5,226.6 L197.1,241.9 L200.6,258.0 L204.1,265.1
 L207.6,255.2 L211.2,268.5 L214.7,263.9 L218.2,268.4 L221.7,271.7 L225.2,274.2 L228.8,273.4 L232.3,279.7
 L235.8,277.7 L239.3,280.0 L242.9,282.6 L246.4,284.8 L249.9,287.0 L253.4,285.1 L257.0,288.1 L260.5,287.6
 L264.0,290.0 L267.5,291.8 L271.0,292.0 L274.6,292.6 L278.1,295.8 L281.6,298.4 L285.1,298.8 L288.7,298.6
 L292.2,300.0 L295.7,299.9 L299.2,301.3 L302.7,301.4 L306.3,302.9 L309.8,304.0 L313.3,304.6 L316.8,305.8
 L320.4,305.7 L323.9,306.3 L327.4,307.0 L330.9,308.2 L334.4,308.9 L338.0,310.1 L341.5,311.1 L345.0,311.1
 L348.5,311.3 L352.1,311.5 L355.6,312.4 L359.1,312.5 L362.6,312.1 L366.1,312.5 L369.7,312.7 L373.2,312.9
 L376.7,312.7 L380.2,313.0 L383.8,313.2 L387.3,313.4 L390.8,313.7 L394.3,313.8 L397.9,313.9 L401.4,313.9
 L404.9,313.8 L408.4,314.1 L411.9,314.1 L415.5,314.1 L419.0,314.1 L422.5,314.2 L426.0,314.2 L429.6,314.2
 L433.1,314.3 L436.6,206.3 L440.1,177.1 L443.6,282.4 L447.2,299.0 L450.7,300.9 L454.2,308.8 L457.7,311.2
 L461.3,306.8 L464.8,313.5 L468.3,316.4 L471.8,315.7 L475.3,323.2 L478.9,326.1 L482.4,319.4 L485.9,324.2
 L489.4,334.6 L493.0,340.6 L496.5,352.9 L500.0,351.7 L503.5,351.4 L507.0,351.2 L510.6,351.4 L514.1,354.7
 L517.6,356.1 L521.1,356.5 L524.7,356.4 L528.2,356.0 L531.7,355.8 L535.2,356.0 L538.8,355.1 L542.3,355.2
 L545.8,355.3 L549.3,284.1 L552.8,345.4 L556.4,349.6 L559.9,353.5 L563.4,364.4 L566.9,367.9 L570.5,372.2
 L574.0,378.3 L577.5,384.0 L581.0,385.3 L584.5,388.2 L588.1,389.9 L591.6,396.1 L595.1,404.3 L598.6,404.4
 L602.2,406.1 L605.7,407.0 L609.2,408.0 L612.7,408.7 L616.2,410.1 L619.8,410.4 L623.3,412.2 L626.8,414.0
 L630.3,413.3 L633.9,413.8 L637.4,414.0 L640.9,415.0 L644.4,415.6 L647.9,415.6 L651.5,416.1 L655.0,416.6
 L658.5,416.7 L662.0,416.9 L665.6,416.9 L669.1,417.0 L672.6,417.1 L676.1,417.4 L679.7,417.4 L683.2,417.4
 L686.7,417.4 L690.2,417.4 L693.7,417.4 L697.3,417.4 L700.8,417.4 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 EnRes X DGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M0.0,81.1 L42.5,81.1 M45.6,60.9 L49.1,66.5 L52.6,69.2

save results to text file

In [50]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [22]:
var ses= sessions.Where(si => si.Name.Contains("iFlux0"));
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:43:47 AM	ff833be6...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:24 AM	041bb15f...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	8/25/2023 10:44:42 AM	1ae3733e...


Number of cells

In [23]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [24]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(ses.Count(),4);


Function used to calculate the EOCs

In [25]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

In [32]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.WriteLine($"P={p},sess={iSess}");
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
        }
    }
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

P=0,sess=0
P=0,sess=1
P=0,sess=2
P=0,sess=3
P=1,sess=0
P=1,sess=1
P=1,sess=2
P=1,sess=3
P=2,sess=0
P=2,sess=1
P=2,sess=2
P=2,sess=3
P=3,sess=0
P=3,sess=1
P=3,sess=2
P=3,sess=3
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

In [33]:
EOCs

[ 0.9038108734340952, 1.921975745497684, 2.3656730498000016, 2.680293477108991 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [28]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlux0"));
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...


In [29]:
var si = sessions.Pick(0);
si

XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...

plot it

In [59]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}